In [ ]:
# Linking with google drive
# simple upload to github, manual here: https://medium.com/analytics-vidhya/how-to-use-google-colab-with-github-via-google-drive-68efb23a42d

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [1]:
%cd /content/drive/My Drive/time_series_analysis/time_series_prediction

/content/drive/My Drive/time_series_analysis/time_series_prediction


In [17]:
#!git clone https://{git_token}@github.com/{username}/{repository}

In [18]:
# trying to import python module as a separate script

In [2]:
import sys
sys.path.append('/content/drive/MyDrive/time_series_analysis/time_series_prediction/dataprocessing/')
sys.path.append('/content/drive/MyDrive/time_series_analysis/time_series_prediction/models/')

In [34]:
import datapreparation
import baselines

In [ ]:
#%load time_series_prediction/models/baselines.py

In [ ]:
#%load time_series_prediction/dataprocessing/datapreparation.py

In [5]:
import pandas as pd

In [8]:
sales_data = pd.read_csv("/content/sales.csv",   header = 0, sep = ";")
categories = pd.read_csv("/content/categories.csv",   header = 0, sep = ";")
traffic = pd.read_csv("/content/traffic.csv",   header = 0, sep = ";")

In [9]:
traffic['DateTime'] = traffic['week_starting_date'].apply(lambda x: pd.to_datetime(str(x), format='%Y%m%d'))
traffic.drop("week_starting_date", axis = 1, inplace = True)

traffic = traffic.set_index(["product_id", "DateTime"])
categories = categories.set_index(["product_id"])

In [10]:
# extended only to global max date horizon
# check nulls here because looks strange
sales_df_combined_right = datapreparation.prepare_sales_data(sales_data, categories, traffic, True, True, [1,2,3])

In [25]:
sales_df_combined_right

sales  category_id  ...  sales_lag_3W  traffic_lag_3W
product_id DateTime                        ...                              
1          2020-07-06    1.0           10  ...           0.0             0.0
           2020-07-13    0.0           10  ...           0.0             0.0
           2020-07-20    1.0           10  ...           0.0             0.0
           2020-07-27    3.0           10  ...           1.0             NaN
           2020-08-03    4.0           10  ...           0.0             NaN
...                      ...          ...  ...           ...             ...
3135       2020-11-30  102.0           43  ...          21.0           100.0
           2020-12-07   45.0           43  ...          29.0            81.0
           2020-12-14    8.0           43  ...         124.0           108.0
           2020-12-21   17.0           43  ...         102.0           142.0
           2020-12-28    7.0           43  ...          45.0            90.0

[217439 rows x 12 columns]

In [ ]:
sample = sales_df_combined_right.loc[[2658, 1308]]

In [12]:
sales_df_combined_right.loc[[2658]].sales[:None]

product_id  DateTime  
2658        2020-02-17     17.0
            2020-02-24     37.0
            2020-03-02     25.0
            2020-03-09    170.0
            2020-03-16    242.0
            2020-03-23    316.0
            2020-03-30    757.0
            2020-04-06    292.0
            2020-04-13    268.0
            2020-04-20    372.0
            2020-04-27    216.0
            2020-05-04    250.0
            2020-05-11    156.0
            2020-05-18    132.0
            2020-05-25    118.0
            2020-06-01     65.0
            2020-06-08     33.0
            2020-06-15     50.0
            2020-06-22     45.0
            2020-06-29     34.0
            2020-07-06     61.0
            2020-07-13     66.0
            2020-07-20     74.0
            2020-07-27     50.0
            2020-08-03     96.0
            2020-08-10     60.0
            2020-08-17     57.0
            2020-08-24     25.0
            2020-08-31      9.0
            2020-09-07     36.0
            2020-

In [ ]:
for i in all_items:
  print("*******************")
  print(i)
  tmp_dataset = sales_df_combined_right.loc[i]
  #n_splits_tmp = int(np.floor((tmp_dataset.shape[0] - 3)/2))
  try:
    tscv = TimeSeriesSplit(n_splits=5, test_size = 3)
    for tr_index, val_index in tscv.split(tmp_dataset):
      print (tr_index, val_index)
      X_tr, X_val = tmp_dataset[["sales"]].iloc[tr_index], tmp_dataset[["sales"]].iloc[val_index]
      print(X_tr)
      print(X_val)
      forecast = get_naive_forecast(X_tr.sales, horizon = 3)
      print(forecast)
      print(mean_absolute_percentage_error(X_val.sales, forecast))
  except Exception as e:
    print(e)

In [ ]:
# securing proper cross validation without errors

In [ ]:
# linear regression

In [ ]:
# hierarchical clustering

### Git commands

In [51]:
!git status

On branch master
Your branch is up to date with 'origin/master'.

Changes not staged for commit:
  (use "git add <file>..." to update what will be committed)
  (use "git checkout -- <file>..." to discard changes in working directory)

	modified:   Master_exploration.ipynb
	modified:   dataprocessing/__pycache__/datapreparation.cpython-37.pyc

no changes added to commit (use "git add" and/or "git commit -a")


In [52]:
!git add .

In [53]:
!git config --global user.email "katarzyna.staniewicz0504@gmail.com"

In [54]:
!git commit -m 'Adding drift'

[master a20a2ee] Adding drift
 2 files changed, 1 insertion(+), 1 deletion(-)
 rewrite Master_exploration.ipynb (88%)


In [55]:
!git push

Counting objects: 6, done.
Delta compression using up to 2 threads.
Compressing objects: 100% (6/6), done.
Writing objects: 100% (6/6), 7.52 KiB | 1.50 MiB/s, done.
Total 6 (delta 2), reused 0 (delta 0)
remote: Resolving deltas: 100% (2/2), completed with 2 local objects.
To https://github.com/katarzynasornat/time_series_prediction
   fa3dfa4..a20a2ee  master -> master


In [ ]:
def NaiveDrift(ts_series, horizon = 3, time_freq = "W-MON"):
  ts_time_index = ts_series.index
  ts_values = ts_series.values

  first_value = ts_values[0]

  try:
    last_value = ts_values[-1]
  except:
    last_time_obs = ts_time_index[0]
    prediciton_time_vector = pd.Index([last_time_obs + pd.offsets.DateOffset(weeks=i) for i in range(1,horizon_for_loop,1)],  freq = "W-MON")
    prediction_series = pd.Series([-999]*horizon, index =prediciton_time_vector)
    return prediction_series

  last_time_obs = ts_time_index[-1]

  slope = round((last_value - first_value)/(len(ts_values) - 1),8)

  horizon_for_loop = horizon + 1
  prediciton_time_vector = pd.Index([last_time_obs + pd.offsets.DateOffset(weeks=i) for i in range(1,horizon_for_loop,1)],  freq = "W-MON")

  prediction_vector = np.linspace(last_value, last_value + slope*horizon, num = horizon_for_loop)[1:]
  prediction_series = pd.Series(prediction_vector, index =prediciton_time_vector)

  return prediction_series


In [44]:
import sklearn
import pandas as pd
import numpy as np

from sklearn.base import BaseEstimator, TransformerMixin, RegressorMixin
from sklearn.utils.validation import check_X_y, check_array, check_is_fitted

In [48]:
model_drift = NaiveDrift(horizon=3)

In [49]:
model_drift.fit(sales_df_combined_right.loc[2658].sales[:-3]).predict(sales_df_combined_right.loc[2658].sales[-3:])

2020-12-14    267.833333
2020-12-21    273.666667
2020-12-28    279.500000
Freq: W-MON, dtype: float64

In [36]:
model_naive = baselines.NaiveForecast(horizon=3)

In [37]:
model_naive.fit(sales_df_combined_right.loc[2658].sales[:-3]).predict(sales_df_combined_right.loc[2658].sales[-3:])

2020-12-14    262.0
2020-12-21    262.0
2020-12-28    262.0
Freq: W-MON, dtype: float64

In [25]:
sales_df_combined_right.loc[2658]

,sales,category_id,traffic,Week_numb,YW,Month,sales_lag_1W,traffic_lag_1W,sales_lag_2W,traffic_lag_2W,sales_lag_3W,traffic_lag_3W
DateTime,,,,,,,,,,,,
2020-02-17,17.0,22,NaN,8,202008,2,0.0,0.0,0.0,0.0,0.0,0.0
2020-02-24,37.0,22,14.0,9,202009,2,17.0,NaN,0.0,0.0,0.0,0.0
2020-03-02,25.0,22,16.0,10,202010,3,37.0,14.0,17.0,NaN,0.0,0.0
2020-03-09,170.0,22,36.0,11,202011,3,25.0,16.0,37.0,14.0,17.0,NaN
2020-03-16,242.0,22,87.0,12,202012,3,170.0,36.0,25.0,16.0,37.0,14.0
2020-03-23,316.0,22,120.0,13,202013,3,242.0,87.0,170.0,36.0,25.0,16.0
2020-03-30,757.0,22,191.0,14,202014,3,316.0,120.0,242.0,87.0,170.0,36.0
2020-04-06,292.0,22,94.0,15,202015,4,757.0,191.0,316.0,120.0,242.0,87.0
2020-04-13,268.0,22,80.0,16,202016,4,292.0,94.0,757.0,191.0,316.0,120.0


In [21]:
sales_df_combined_right.loc[[2658]].sales[:-3]

product_id  DateTime  
2658        2020-02-17     17.0
            2020-02-24     37.0
            2020-03-02     25.0
            2020-03-09    170.0
            2020-03-16    242.0
            2020-03-23    316.0
            2020-03-30    757.0
            2020-04-06    292.0
            2020-04-13    268.0
            2020-04-20    372.0
            2020-04-27    216.0
            2020-05-04    250.0
            2020-05-11    156.0
            2020-05-18    132.0
            2020-05-25    118.0
            2020-06-01     65.0
            2020-06-08     33.0
            2020-06-15     50.0
            2020-06-22     45.0
            2020-06-29     34.0
            2020-07-06     61.0
            2020-07-13     66.0
            2020-07-20     74.0
            2020-07-27     50.0
            2020-08-03     96.0
            2020-08-10     60.0
            2020-08-17     57.0
            2020-08-24     25.0
            2020-08-31      9.0
            2020-09-07     36.0
            2020-